## Langchain 라이브러리에서 Chunking을 지원하는 클래스와 함수들을 알아보자
- CharacterTextSplitter
  - 텍스트를 지정된 기준으로 나눠주는 기본적인 클래스
  - 구분자는 디폴트로 "\n\n" 을 사용하지만 변경 가능
  - 구분자가 단일 문자이기 때문에 연속된 텍스트의 의미를 충분히 고려하지 못할 수 있음

- RecursiveCharacterTextSplitter
  - 텍스트를 여러 단계를 거쳐 나누어주는 클래스(청킹에서 가장 많이 활용됨)
  - 리스트를 통해서 구분자를 여러개 사용하여 세분화된 청크로 만들 수 있음(구분자의 순서가 중요함)
  - 처음엔 "\n\n"으로 분리하고 청크가 여전히 크다면 "\n", 그래도 크다면 " " 이런 식으로 지정된 크기 제한에 들어올 때까지 계층적 분할을 진행
  - 계층적 분할을 통해 의미가 있는 단위가 가능한 온전하게 유지되도록 함

In [1]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [2]:
text1 = "청킹을 위한 예제 데이터 입니다. 어떻게 쪼개지는지 알아볼까요?"
text2 = "청킹을 위한 예제 데이터 입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다."
text3 = """청킹을 위한 예제 데이터 입니다. 
어떻게 쪼개지는지 알아볼까요?

결과를 확인해봅시다."""

### Character Splitter

In [3]:
# 청킹 객체 생성
chunk_size = 20    # 하나의 청크에 대한 길이 기준을 문자20개로 설정
chunk_overlap = 0

c_splitter = CharacterTextSplitter(
    # 현재는 예시 문장이 엄청 짧아서 20문자 정도로 지정했지만 일반적인 문서에서는 200~1500사이로 지정
    # 데이터(문서)의 종류에 따라 달라짐(기술 문서나 연구논문은 상대적으로 큰게 좋고, 일반 문서는 작은게 좋음)
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [4]:
# split_text : 텍스트 데이터 청킹 실행 함수
c_splitter.split_text(text1)

['청킹을 위한 예제 데이터 입니다. 어떻게 쪼개지는지 알아볼까요?']

In [5]:
c_splitter.split_text(text2)

['청킹을 위한 예제 데이터 입니다. 어떻게 쪼개지는지 알아볼까요? 결과를 확인해봅시다.']

In [6]:
c_splitter.split_text(text3)

# 무조건 Chunk_size에 맞게 분리가 되는 것은 아니고 의미 있는 단위라면 더 길거나 짧게 출력될 수 있음

Created a chunk of size 36, which is longer than the specified 20


['청킹을 위한 예제 데이터 입니다. \n어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

### Recursive Splitter

In [7]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [8]:
r_splitter.split_text(text1)

['청킹을 위한 예제 데이터 입니다.', '어떻게 쪼개지는지 알아볼까요?']

In [9]:
r_splitter.split_text(text2)

['청킹을 위한 예제 데이터 입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

In [10]:
r_splitter.split_text(text3)

['청킹을 위한 예제 데이터 입니다.', '어떻게 쪼개지는지 알아볼까요?', '결과를 확인해봅시다.']

### Character, Recursive 비교

In [11]:
see_you_again = """It's been a long day without you, my friend.
And I'll tell you all about it when I see you again.
We've come a long way from where we began.
Oh, I'll tell you all about it when I see you again.
When I see you again.


Damn, who knew?
All the planes we flew, good things we been through.
That I'd be standing right here talking to you.
'Bout another path, I know we loved to hit the road and laugh.
But something told me that it wouldn't last.
Had to switch up, look at things different, see the bigger picture.
Those were the days, hard work forever pays.
Now I see you in a better place (see you in a better place).
Uh."""

In [12]:
len(see_you_again)

620

In [13]:
c_splitter = CharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    separator = '\n\n'   # 청크로 나누는 기준(직접 지정 가능)
)

c_splitter.split_text(see_you_again)

Created a chunk of size 215, which is longer than the specified 100


["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.\nWe've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.\nWhen I see you again.",
 "Damn, who knew?\nAll the planes we flew, good things we been through.\nThat I'd be standing right here talking to you.\n'Bout another path, I know we loved to hit the road and laugh.\nBut something told me that it wouldn't last.\nHad to switch up, look at things different, see the bigger picture.\nThose were the days, hard work forever pays.\nNow I see you in a better place (see you in a better place).\nUh."]

In [14]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    separators = ["\n\n", "\n", " "]   # 리스트로 여러 조건을 설정가능(계층적이므로 순서가 중요함)
)
r_splitter.split_text(see_you_again)

["It's been a long day without you, my friend.\nAnd I'll tell you all about it when I see you again.",
 "We've come a long way from where we began.\nOh, I'll tell you all about it when I see you again.",
 'When I see you again.',
 'Damn, who knew?\nAll the planes we flew, good things we been through.',
 "That I'd be standing right here talking to you.",
 "'Bout another path, I know we loved to hit the road and laugh.",
 "But something told me that it wouldn't last.",
 'Had to switch up, look at things different, see the bigger picture.',
 'Those were the days, hard work forever pays.',
 'Now I see you in a better place (see you in a better place).\nUh.']

- CharacterTextSplitter를 사용한 결과가 만족스럽지 못하다면 더 작은 단위의 구분자를 지정하여 해결하거나 Recursive를 사용하여 계층적 분할을 적용하면 됨
- 처음부터 Recursive를 사용하는것이 더 편리할 수 있음

### 웹 페이지에 접근해 텍스트 데이터 가져오기

In [15]:
# WebBaseLoader : langchain라이브러리에서 지원하는 웹 페이지 HTML 컨텐츠를 로드해주는 클래스
#                 (내부적으로 BeautifulSoup을 사용함)
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [17]:
loader = WebBaseLoader("https://n.news.naver.com/mnews/article/003/0013049285")

# 브라우저 우회 접속
loader.requests_kwargs = {'headers': {'User-Agent': 'Mozilla/5.0'}}
# 한글 인코딩 설정
loader.encoding = 'utf-8'

docs = loader.load()
docs

[Document(metadata={'source': 'https://n.news.naver.com/mnews/article/003/0013049285', 'title': '"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"', 'language': 'ko'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n본문 바로가기\n\n\n\n\n\n\nNAVER\n\n뉴스\n\n\n엔터\n\n\n\n\n스포츠\n\n\n\n\n날씨\n\n\n\n\n프리미엄\n\n\n\n\n\n\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n언론사별\n\n\n정치\n\n\n경제\n\n\n사회\n\n\n생활/문화\n\n\nIT/과학\n\n\n세계\n\n\n랭킹\n\n\n신문보기\n\n\n오피니언\n\n\nTV\n\n\n팩트체크\n\n\n알고리즘 안내\n\n\n정정보도 모음\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n뉴시스\n\n뉴시스\n\n\n구독\n\n뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n뉴시스 언론사 구독 해지되었습니다.\n\n\n\n\nPICK\n안내\n\n\n언론사가 주요기사로선정한 기사입니다.\n언론사별 바로가기\n닫기\n\n\n\n\n"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"\n\n\n\n\n입력2025.02.05. 오후 5:29\n\n\n수정2025.02.05. 오후 5:41\n\n기사원문\n \n\n\n\n\n최현호 기자\n\nTALK\n\n\n\n\n\n\n\n\n\n최현호 기자\n\nTALK\n구독\n구독중\n\n\n\n\n구독자\n0\n\n\n응원수\n0\n\n\n\n더보기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감

In [19]:
print(docs[0].page_content[:500])





















"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"














본문 바로가기






NAVER

뉴스


엔터




스포츠




날씨




프리미엄










검색










언론사별


정치


경제


사회


생활/문화


IT/과학


세계


랭킹


신문보기


오피니언


TV


팩트체크


알고리즘 안내


정정보도 모음
















뉴시스

뉴시스


구독

뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.
보러가기


뉴시스 언론사 구독 해지되었습니다.




PICK
안내


언론사가 주요기사로선정한 기사입니다.
언론사별 바로가기
닫기




"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"




입력2025.02.05. 오후 5:29


수정2025.02.05. 오후 5:41

기사원문
 




최현호 기자

TALK










- 문자열 처리 코드나 정규표현식(re) 등을 활용하여 필요한 텍스트만 추출해서 사용 가능

#### 1) 문자열 함수 replace 활용

In [20]:
docs[0].page_content[:500].replace("\n", "")

'"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"본문 바로가기NAVER뉴스엔터스포츠날씨프리미엄검색언론사별정치경제사회생활/문화IT/과학세계랭킹신문보기오피니언TV팩트체크알고리즘 안내정정보도 모음뉴시스뉴시스구독뉴시스 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.보러가기뉴시스 언론사 구독 해지되었습니다.PICK안내언론사가 주요기사로선정한 기사입니다.언론사별 바로가기닫기"작년 금 수요 사상 최고치 기록…불확실성 심화 영향"입력2025.02.05. 오후 5:29수정2025.02.05. 오후 5:41기사원문 최현호 기자TALK'

#### 2) 문자열 정규표현식 활용
- 정규표현식은 검색 결과가 개별 문자열로 출력되기 때문에 필요에 따라 이를 합쳐서 전체 텍스트로 활용할 수 있음

In [21]:
import re

In [22]:
# 문자열 앞의 r의 의미는 문자열 내에 이스케이프 코드를 지칭하는 \(역슬래시)가 있는 경우 이를 문자열로 인식하게함
# [\w] : 한글, 영문대소문자, 숫자 및 언더바를 포함하는 텍스트 추출
# + : 연속된 문자들을 문자열로 반환
re.findall(r"[\w]+", docs[0].page_content[:500])

['작년',
 '금',
 '수요',
 '사상',
 '최고치',
 '기록',
 '불확실성',
 '심화',
 '영향',
 '본문',
 '바로가기',
 'NAVER',
 '뉴스',
 '엔터',
 '스포츠',
 '날씨',
 '프리미엄',
 '검색',
 '언론사별',
 '정치',
 '경제',
 '사회',
 '생활',
 '문화',
 'IT',
 '과학',
 '세계',
 '랭킹',
 '신문보기',
 '오피니언',
 'TV',
 '팩트체크',
 '알고리즘',
 '안내',
 '정정보도',
 '모음',
 '뉴시스',
 '뉴시스',
 '구독',
 '뉴시스',
 '언론사',
 '구독되었습니다',
 '메인',
 '뉴스판에서',
 '주요뉴스를',
 '볼',
 '수',
 '있습니다',
 '보러가기',
 '뉴시스',
 '언론사',
 '구독',
 '해지되었습니다',
 'PICK',
 '안내',
 '언론사가',
 '주요기사로선정한',
 '기사입니다',
 '언론사별',
 '바로가기',
 '닫기',
 '작년',
 '금',
 '수요',
 '사상',
 '최고치',
 '기록',
 '불확실성',
 '심화',
 '영향',
 '입력2025',
 '02',
 '05',
 '오후',
 '5',
 '29',
 '수정2025',
 '02',
 '05',
 '오후',
 '5',
 '41',
 '기사원문',
 '최현호',
 '기자',
 'TALK']

In [23]:
" ".join(re.findall(r"[\w]+", docs[0].page_content))

'작년 금 수요 사상 최고치 기록 불확실성 심화 영향 본문 바로가기 NAVER 뉴스 엔터 스포츠 날씨 프리미엄 검색 언론사별 정치 경제 사회 생활 문화 IT 과학 세계 랭킹 신문보기 오피니언 TV 팩트체크 알고리즘 안내 정정보도 모음 뉴시스 뉴시스 구독 뉴시스 언론사 구독되었습니다 메인 뉴스판에서 주요뉴스를 볼 수 있습니다 보러가기 뉴시스 언론사 구독 해지되었습니다 PICK 안내 언론사가 주요기사로선정한 기사입니다 언론사별 바로가기 닫기 작년 금 수요 사상 최고치 기록 불확실성 심화 영향 입력2025 02 05 오후 5 29 수정2025 02 05 오후 5 41 기사원문 최현호 기자 TALK 최현호 기자 TALK 구독 구독중 구독자 0 응원수 0 더보기 추천 쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0 댓글 본문 요약봇 본문 요약봇도움말 자동 추출 기술로 요약된 내용입니다 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다 닫기 텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름 이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다 본문듣기 시작 닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기 지정학적 경제적 불확실성 심화 분기 최고치도 달성 서울 뉴시스 정병혁 기자 전 세계 금 수요가 중앙은행의 강력한 매입과 투자 수요 증가에 힘입어 지난해 또 다시 역대 최고치를 기록했다고 세계금협의회 WGC 가 연례 보고서를 통해 밝혔다 사진은 5일 서울 종로구 한국금거래소 종로본점 모니터에 금 시세가 나타나 있는 모습 2025 2 5 서울 뉴시스 최현호 기자 전 세계 금 수요가 중앙은행의 강력한 매입과 투자 수요 증가에 힘입어 지난해 또 다시 역대 최고치를 기록했다고 세계금협의회 WGC 가 연례 보고서를 통해 밝혔다 지난해 총 금 거래량은 장외거래 OTC 

### 문서 로더(Document Loader)
- Langchain의 문서로더를 사용해 다양한 형식의 데이터 파일로부터 텍스트를 로드할 수 있음
- 로드한 문서는 Document 객체로 변환되어 출력되며, page_content에는 내용, metadata에는 데이터에 대한 설명을 포함하고 있음
- 간단한 PDF파일을 로드하거나 Word, CSV, Excel, JSON 등을 로드하기 위한 문서 로더들이 각각 존재함

#### 문서 로더 종류
- TextLoader : txt파일 로드
- PyPDFLoader : PDF파일 로드
- JSONLoader : JSON파일 로드
- CSVLoader : CSV파일 로드
- UnstructuredHTMLLoader : HTML파일 로드

#### TextLoader

In [24]:
from langchain.document_loaders import TextLoader

In [27]:
loader = TextLoader('data/멜론차트 TOP100.txt', encoding='utf-8')

doc = loader.load()
print(doc[0].page_content)

	타이틀	아티스트
0	비의 랩소디	임재현
1	To. X	태연 (TAEYEON)
2	Perfect Night	LE SSERAFIM (르세라핌)
3	Drama	aespa
4	에피소드	이무진
5	사랑은 늘 도망가	임영웅
6	헤어지자 말해요	박재정
7	그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))	너드커넥션 (Nerd Connection)
8	인사	범진
9	Love 119	RIIZE
10	Seven (feat. Latto) - Clean Ver.	정국
11	Baddie	IVE (아이브)
12	우리들의 블루스	임영웅
13	모래 알갱이	임영웅
14	Do or Die	임영웅
15	다시 만날 수 있을까	임영웅
16	첫 눈	EXO
17	I AM	IVE (아이브)
18	Love Lee	AKMU (악뮤)
19	이제 나만 믿어요	임영웅
20	Get A Guitar	RIIZE
21	무지개	임영웅
22	You & Me	제니 (JENNIE)
23	London Boy	임영웅
24	아버지	임영웅
25	Polaroid	임영웅
26	Standing Next to You	정국
27	인생찬가	임영웅
28	너의 모든 순간	성시경
29	후라이의 꿈	AKMU (악뮤)
30	A bientot	임영웅
31	Super Shy	NewJeans
32	사막에서 꽃을 피우듯	우디 (Woody)
33	Ditto	NewJeans
34	잘 지내자, 우리 (여름날 우리 X 로이킴)	로이킴
35	ETA	NewJeans
36	Hype Boy	NewJeans
37	연애편지	임영웅
38	퀸카 (Queencard)	(여자)아이들
39	어떻게 이별까지 사랑하겠어, 널 사랑하는 거지	AKMU (악뮤)
40	MANIAC	VIVIZ (비비지)
41	별 떨어진다 (I Do)	디오 (D.O.)
42	사랑인가 봐	멜로망스
43	Discord	QWER
44	Dynamite	방탄소년단
45	봄날	방탄소년단
46	Spicy	aespa
47	사건의 지평선	윤하 (YOUNHA)
48	사랑할 수밖에	볼빨간사춘기
49	사랑하

In [28]:
r_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                            chunk_overlap = 0
                                           )

# load_and_split : 데이터를 로드하며 동시에 설정된 splitter를 적용해주는 수
docs_split1 = loader.load_and_split(r_splitter)
print(f"청크 개수: {len(docs_split1)}개")
print()

for doc in docs_split1 :
    print(doc.page_content)
    print("="*100)

청크 개수: 5개

타이틀	아티스트
0	비의 랩소디	임재현
1	To. X	태연 (TAEYEON)
2	Perfect Night	LE SSERAFIM (르세라핌)
3	Drama	aespa
4	에피소드	이무진
5	사랑은 늘 도망가	임영웅
6	헤어지자 말해요	박재정
7	그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))	너드커넥션 (Nerd Connection)
8	인사	범진
9	Love 119	RIIZE
10	Seven (feat. Latto) - Clean Ver.	정국
11	Baddie	IVE (아이브)
12	우리들의 블루스	임영웅
13	모래 알갱이	임영웅
14	Do or Die	임영웅
15	다시 만날 수 있을까	임영웅
16	첫 눈	EXO
17	I AM	IVE (아이브)
18	Love Lee	AKMU (악뮤)
19	이제 나만 믿어요	임영웅
20	Get A Guitar	RIIZE
21	무지개	임영웅
22	You & Me	제니 (JENNIE)
23	London Boy	임영웅
24	아버지	임영웅
25	Polaroid	임영웅
26	Standing Next to You	정국
27	인생찬가	임영웅
28	너의 모든 순간	성시경
29	후라이의 꿈	AKMU (악뮤)
30	A bientot	임영웅
31	Super Shy	NewJeans
32	사막에서 꽃을 피우듯	우디 (Woody)
33	Ditto	NewJeans
34	잘 지내자, 우리 (여름날 우리 X 로이킴)	로이킴
35	ETA	NewJeans
36	Hype Boy	NewJeans
37	연애편지	임영웅
38	퀸카 (Queencard)	(여자)아이들
39	어떻게 이별까지 사랑하겠어, 널 사랑하는 거지	AKMU (악뮤)
40	MANIAC	VIVIZ (비비지)
41	별 떨어진다 (I Do)	디오 (D.O.)
42	사랑인가 봐	멜로망스
43	Discord	QWER
44	Dynamite	방탄소년단
45	봄날	방탄소년단
46	Spicy	aespa
47	사건의 지평선	윤하 (YOUNHA)
48	사랑할 수밖에	볼빨간

#### PyPDFLoader

In [29]:
# 파이썬용 PDF 처리 라이브러리 설치
!pip install pypdf

In [30]:
from langchain.document_loaders import PyPDFLoader

In [31]:
loader = PyPDFLoader("data/기술보증기금과 한국경제.pdf")

# load_and_split : PDF로드 후 페이지 별로 분리
docs = loader.load_and_split()

print("페이지 수 :", len(docs))
docs

페이지 수 : 5


[Document(metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조) \n✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증 \n 신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하는 종업원이 1천명 이하이고 총자산액이 1천억원 이하인 기업이 부담하는 금전\n채무 보증. \n기술평가 해당 기술과 관련된 기술성·시장성·사업타당성 등을 종합적으로 평가하여 \n금액·등급·의견 또는 점수 등으로 표시하는 것을 말한다 \n* 기금은 기술평가의 객관성 및 공정성 등을 확보하기 위하여 기술평가의 \n기준·절차·방법·종류 등에 관한 사항을 미리 정하여야 

In [32]:
pdf_splitter = RecursiveCharacterTextSplitter(chunk_size = 300,
                                              chunk_overlap = 0
                                             )

docs_split = loader.load_and_split(pdf_splitter)
print(f"청크 개수: {len(docs_split)}개")
docs_split

청크 개수: 22개


[Document(metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='페이지 1 / 5 \n \n기술보증기금과 한국경제 \n \nI. 기술보증기금 개요  \n1. 설립근거 : 기술보증기금법 \n \n- 설립목적(존재이유) \n✓ 기술보증기금을 설립하여 기술보증제도를 정착·발전시킴으로써 신 기술사업에 대한 자금의 \n공급을 원활하게 하고 나아가 국민 경제의 발전에 이바지함을 목적으로 함(기술보증기금법1\n조) \n✓ 설립 : 담보능력이 미약한 기업의 채무를 보증하게 하여 기업에 대한 자금 융통을 원활하게 하기 \n위하여 기술보증기금을 설립(법 12조)'),
 Document(metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='✓ 기금의 재원 : 정부 출연금의 예산은 중소벤처기업부 소관으로 함 \n \n☞ 기술보증기금은  \n✓ 기술력은 우수하지만 담보 부족한 중소기업의 \n✓ 기술성과 사업성 평가를 통해 기술보증을 지원하며, \n✓ 기술평가, 벤처이노비즈기업  인증, 중소기업 창업지원 등의 업무 수행 \n \n2. 주요개념1 \n \n업  무 내  용 \n기술보증 신기술사업자가 부담하는 금전 채무 보증.( 신용부족-담보부족 해결) \n \n금융회사 등으로부터 자금 대출을 받음으로써 금융회사 등에 대하여 부담하는 금전 채무를 \n기술보증기금이 기술보증서로 보증'),
 Document(metadata={'source': 'data/기술보증기금과 한국경제.pdf', 'page': 0, 'page_label': '1'}, page_content='신기술사업자 - 기술을 개발하거나 이를 응용하여 사업화하는 중소기업(「중소기업기본법」에 \n따른 중소기업) 및 대통령령으로 정하는 기업 \n- "기업"이란 사업을 하는 개인 및 법인 \n신용보증 상시 사용하는 종

#### JSONLoader

In [33]:
# JSON 데이터 추출 및 기타 작업들을 지원해주는 라이브러리
!pip install jq

In [34]:
from langchain.document_loaders import JSONLoader

In [38]:
loader = JSONLoader("data/ya_review_glad.json",
                    # jq_schema : JSON 구조에서 특정 부분을 추출하는데 사용하는 인자
                    jq_schema = ".[].review"   # 현재는 리스트 내에 key가 review인 값들만 추출하겠다는 뜻
                   )
doc = loader.load()
doc

[Document(metadata={'source': 'C:\\Users\\CHOI\\새싹 종로 3기\\data\\ya_review_glad.json', 'seq_num': 1}, page_content='편리하고 깨끗한 시설이 좋았습니다'),
 Document(metadata={'source': 'C:\\Users\\CHOI\\새싹 종로 3기\\data\\ya_review_glad.json', 'seq_num': 2}, page_content='가족과 함께 묵었습니다. 아주 편안하게 이용했습니다.\n침대도 부드럽고 청결하고, 욕실도 깨끗했습니다.'),
 Document(metadata={'source': 'C:\\Users\\CHOI\\새싹 종로 3기\\data\\ya_review_glad.json', 'seq_num': 3}, page_content='체크인\n체크인 시간보다 일찍 가도 얼리 체크인이 가능해서 빠르게 일정을 준비할 수 있어서 좋았습니다.\n방 컨디션\n방은 호텔이라는 이름에 맞게 상당히 깔끔하게 준비되어 있었습니다 크기는 2명에 사용하기 적당하다고 생각되고 환기가 가능한 창문이 있어서 좋았지만 뷰는 일반적인 도로뷰 입니다.\n침구, 서비스\n침구는 상당히 좋았던 경험입니다 숙박기간동안 잠을 정말 잘잤으며 이불도 푹신하고 깔끔해서 좋은 기분으로 잠을 잘 수 있었습니다 호텔리어분들의 친철함과 캐리어 보관 서비스 또한 정말 좋고 로봇을 활용한 서비스와 특별한 경험을 했습니다\n기타 \n룸서비스는 따로 없지만 지하에 일식집과 양식집이 있어서 이용하기 좋았고 바로 앞에 삼성역이 있어서 교통적으로 이점과 코엑스가 가까워 상당한 지리적인 이점이 있다고 생각되는곳입니다~'),
 Document(metadata={'source': 'C:\\Users\\CHOI\\새싹 종로 3기\\data\\ya_review_glad.json', 'seq_num': 4}, page_content='침구류가 아주 좋네요'),
 Document(metadata={'source'

In [39]:
docs_split = loader.load_and_split(r_splitter)
print(f"청크 개수: {len(docs_split)}개")
print()

for doc in docs_split :
    print(doc.page_content)
    print("="*100)

청크 개수: 420개

편리하고 깨끗한 시설이 좋았습니다
가족과 함께 묵었습니다. 아주 편안하게 이용했습니다.
침대도 부드럽고 청결하고, 욕실도 깨끗했습니다.
체크인
체크인 시간보다 일찍 가도 얼리 체크인이 가능해서 빠르게 일정을 준비할 수 있어서 좋았습니다.
방 컨디션
방은 호텔이라는 이름에 맞게 상당히 깔끔하게 준비되어 있었습니다 크기는 2명에 사용하기 적당하다고 생각되고 환기가 가능한 창문이 있어서 좋았지만 뷰는 일반적인 도로뷰 입니다.
침구, 서비스
침구는 상당히 좋았던 경험입니다 숙박기간동안 잠을 정말 잘잤으며 이불도 푹신하고 깔끔해서 좋은 기분으로 잠을 잘 수 있었습니다 호텔리어분들의 친철함과 캐리어 보관 서비스 또한 정말 좋고 로봇을 활용한 서비스와 특별한 경험을 했습니다
기타 
룸서비스는 따로 없지만 지하에 일식집과 양식집이 있어서 이용하기 좋았고 바로 앞에 삼성역이 있어서 교통적으로 이점과 코엑스가 가까워 상당한 지리적인 이점이 있다고 생각되는곳입니다~
침구류가 아주 좋네요
주변 활용도가 높습니다
명불허전 글래드 최고입니다!
위치가 너무  좋은 숙소 입니다. 다음날 출국이라 고른 숙소! 깨끗하고  1층에 조용히 이야기 나눌 곳도 있는 좋은 호텔 . 강추입니다.
가갹대비 편리하고 좋아요
글래드삼성 위치도 좋고 무엇보다 침구가 너무 편안합니다. :) 편안한 숙박이었어요.
응대도 친절합니다.
3층으로 배정받았는데 커텐치면 도로에 차들과 지나가는 사람들이랑 아이컨텍 할정도록 너무 훤히 다 보여서
커텐을 열어둘수가 없었어요ㅠ.ㅠ
코엑스랑 가까워서 좋구요 
방 배정해주신 직원분은 피곤하신건지 친절하시다는 느낌은 못받았숩니다.
발렛주차 직원 너무 친절하고 편리합니다. 감사합니다
방이 그다지 넓진 않고
삼성역이 장기공사중이라 뷰도 기대할 건 없지만
체크인이 간편하고 침구도 깔끔하고
요청사항도 잘 반영해주셔서 편안하게 이용했습니다
편안하게 잘 사용했습니다.
항상 호캉스 하고싶을때 가요! 지하에 있는 식당들도 맛있고, 코엑스나 맛집들과도 가까워서 편해요! 방 상